## Notebook for calculating various molecular desciptors

In [104]:
# Imports
from rdkit import Chem, RDLogger
from rdkit.Chem import AllChem
import os
from mordred import Calculator, descriptors
import pandas as pd
from more_itertools import chunked
from tqdm.notebook import tqdm
import numpy as np
# Capturing RDKit errors
from io import StringIO
import sys
# For DScribe descriptors
from ase import io
from dscribe import descriptors
from collections import defaultdict

In [59]:
def mol2_to_mol(file=None, sanitize=True):
    mols={}
    with open(file, 'r') as f:
        line =f.readline()
        # Counter represents the unique index for molecules.
        counter = 0
        while not f.tell() == os.fstat(f.fileno()).st_size:
            if line.startswith("@<TRIPOS>MOLECULE"):
                mol = []
                mol.append(line)
                line = f.readline()
                while not line.startswith("@<TRIPOS>MOLECULE"):
                    mol.append(line)
                    line = f.readline()
                    if f.tell() == os.fstat(f.fileno()).st_size:
                        mol.append(line)
                        break
                mol[-1] = mol[-1].rstrip() # removes blank line at file end
                block = ",".join(mol).replace(',','')
                m=Chem.MolFromMol2Block(block, sanitize=sanitize, removeHs=False)
            if m is not None:
                mols[counter] = m
            counter += 1
    return mols

In [60]:
def parse_molecules(mols):
    # Removes molecules that generate warnings when parsing through RDKit
    # This could lead to incorrect values depending on how the descriptor
    # is calculated.
    working_mols = {}
    nonworking_mols = {}
    sio = sys.stderr = StringIO()
    for idx, mol in mols.items():
        Chem.SanitizeMol(mol)
        res = sio.getvalue()
        if 'WARNING' in res:
            nonworking_mols[idx] = mol
            print(res)
            # Reset stderr
            sio = sys.stderr = StringIO()
        else:
            working_mols[idx] = mol
            sio = sys.stderr = StringIO()
    return working_mols, nonworking_mols

def calculate_descriptors_pandas(mols, check_mols=True):
    # Calculates 2D and 3D descriptors using Mordred.
    # Returns a DataFrame containing descriptors.
    calc = Calculator(descriptors, ignore_3D=False)
    if check_mols:
        mols, _ = parse_molecules(mols)
    df = calc.pandas(list(mols.values()), ipynb=True, quiet=False)
    df.index.name = 'Molecule_Number'
    df['SMILES'] = [Chem.MolToSmiles(m) for m in list(mols.values())]
    df['RDKit_Molecule'] = [m for m in list(mols.values())]
    df.index = list(mols.keys())
    return df

In [61]:
mols = mol2_to_mol('../small_molecule_search.mol2')
df = calculate_descriptors_pandas(mols, check_mols=True)

RDKit WARNING: [21:56:55] sanitise [21:56:55] 01: [21:56:56] WARNING: could not find number of expected rings. Switching to an approximate ring finding algorithm.

RDKit WARNING: [21:56:56] WARNING: could not find number of expected rings. Switching to an approximate ring finding algorithm.

RDKit WARNING: [21:56:56] WARNING: could not find number of expected rings. Switching to an approximate ring finding algorithm.

RDKit WARNING: [21:56:56] WARNING: could not find number of expected rings. Switching to an approximate ring finding algorithm.

RDKit WARNING: [21:56:56] WARNING: could not find number of expected rings. Switching to an approximate ring finding algorithm.

RDKit WARNING: [21:56:56] WARNING: could not find number of expected rings. Switching to an approximate ring finding algorithm.

RDKit WARNING: [21:56:56] WARNING: could not find number of expected rings. Switching to an approximate ring finding algorithm.

RDKit WARNING: [21:56:56] WARNING: could not find number of ex

  0%|          | 0/29079 [00:00<?, ?it/s]

In [22]:
df.to_pickle('Calculated_Descriptors.pkl')

In [7]:
df_pickled = pd.read_pickle('Calculated_Descriptors.pkl')

In [62]:
df = pd.DataFrame(df)

In [63]:
def calculate_fingerprints(mols):
    # Converts an RDKit molecule into a fingerprint. 
    # This function is only given as an example of a fingerprint calculation.
    # There are multiple different approaches to calculate molecular fingerprints.
    # A radius 2 and bit length of 1024 was chosen for this calculations.
    fps = []
    for mol in mols:
        fp = np.array(AllChem.GetMorganFingerprintAsBitVect(mol, radius=2, nBits=1024))
        fps.append(fp)
    return fps

In [64]:
df['Fingerprints'] = calculate_fingerprints(df['RDKit_Molecule'])

In [132]:
# Calculating DScribe descriptors
def calculate_dscribe_descriptors(mols):
    dscribe_descriptors = defaultdict(list)
    for mol in tqdm(mols):
        mol_block = StringIO(Chem.MolToMolBlock(mol))
        ase_mol = ase.io.mol.read_mol(mol_block)
        unique_species = list(set(ase_mol.get_chemical_symbols()))
        # Setting up the DScribe descriptors
        # The parameters for all these descriptors are unoptimised.
        cm = descriptors.CoulombMatrix(
            n_atoms_max=1000,
        )
        cm_result = cm.create(ase_mol, n_jobs=-1)
        sm = descriptors.SineMatrix(
            n_atoms_max=1000,
            permutation="sorted_l2",
            sparse=False,
            flatten=True
        )
        sm_result = cm.create(ase_mol, n_jobs=-1)
        dscribe_descriptors['Coulomb_Matrix'].append(cm_result)
        dscribe_descriptors['Sine_Matrix'].append(sm_result)
    return dscribe_descriptors

    

In [133]:
dscribe_descriptors = calculate_dscribe_descriptors(df['RDKit_Molecule'].to_list())

  0%|          | 0/29079 [00:00<?, ?it/s]

In [135]:
pd.DataFrame(dscribe_descriptors)

,Coulomb_Matrix,Sine_Matrix
0,"[[73.51669471981023, 20.951345335276688, 9.005...","[[73.51669471981023, 20.951345335276688, 9.005..."
1,"[[73.51669471981023, 22.856319067860767, 28.74...","[[73.51669471981023, 22.856319067860767, 28.74..."
2,"[[388.0234410266618, 89.20065413411893, 89.092...","[[388.0234410266618, 89.20065413411893, 89.092..."
3,"[[73.51669471981023, 15.43121044030552, 25.528...","[[73.51669471981023, 15.43121044030552, 25.528..."
4,"[[73.51669471981023, 9.361924757951526, 8.9201...","[[73.51669471981023, 9.361924757951526, 8.9201..."
...,...,...
29074,"[[36.85810519942594, 24.78187166552364, 26.537...","[[36.85810519942594, 24.78187166552364, 26.537..."
29075,"[[36.85810519942594, 25.683059094169057, 14.98...","[[36.85810519942594, 25.683059094169057, 14.98..."
29076,"[[36.85810519942594, 25.574951394660154, 25.97...","[[36.85810519942594, 25.574951394660154, 25.97..."
29077,"[[36.85810519942594, 25.550946068413886, 26.02...","[[36.85810519942594, 25.550946068413886, 26.02..."
